In [ ]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy as sp
import pyamg
import random
import math
from torch.autograd import Variable
from pyamg.multilevel import multilevel_solver
from pyamg.relaxation.smoothing import change_smoothers
from scipy.sparse import csr_matrix, coo_matrix, lil_matrix, isspmatrix_csr, SparseEfficiencyWarning
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device:', device)

def diffusion_stencil_2d(epsilon=1.0, theta=0.0, type='FE'):
    eps = float(epsilon)  # for brevity
    theta = float(theta)

    C = np.cos(theta)
    S = np.sin(theta)
    CS = C*S
    CC = C**2
    SS = S**2

    if(type == 'FE'):
        a = (-1*eps - 1)*CC + (-1*eps - 1)*SS + (3*eps - 3)*CS
        b = (2*eps - 4)*CC + (-4*eps + 2)*SS
        c = (-1*eps - 1)*CC + (-1*eps - 1)*SS + (-3*eps + 3)*CS
        d = (-4*eps + 2)*CC + (2*eps - 4)*SS
        e = (8*eps + 8)*CC + (8*eps + 8)*SS

        stencil = np.array([[a, b, c],
                            [d, e, d],
                            [c, b, a]]) / 6.0

    elif type == 'FD':

        a = -0.5*(eps - 1)*CS
        b = -(eps*SS + CC)
        c = -a
        d = -(eps*CC + SS)
        e = 2.0*(eps + 1)

        stencil = np.array([[a+c, d-2*c, 2*c],
                            [b-2*c, e+4*c, b-2*c],
                            [2*c, d-2*c, a+c]])


    return stencil
def map_2_to_1(grid_size_x=8,grid_size_y=8,stencil_size=3):
    # maps 2D coordinates to the corresponding 1D coordinate in the matrix.
    k = np.zeros((grid_size_x, grid_size_y, stencil_size, stencil_size))
    M = np.reshape(np.arange(grid_size_x * grid_size_y), (grid_size_x, grid_size_y))
    M = np.concatenate([M, M], 0)
    M = np.concatenate([M, M], 1)
    for i in range(stencil_size):
        I = (i - stencil_size//2) % grid_size_x
        for j in range(stencil_size):
            J = (j - stencil_size//2) % grid_size_y
            k[:, :, i, j] = M[I:I + grid_size_x, J:J + grid_size_y]
    return k
def get_p_matrix_indices_one(grid_size):
    K = map_2_to_1(grid_size=grid_size)
    indices = []
    for ic in range(grid_size // 2):
        i = 2 * ic + 1
        for jc in range(grid_size // 2):
            j = 2 * jc + 1
            J = int(grid_size // 2 * jc + ic)
            for k in range(3):
                for m in range(3):
                    I = int(K[i, j, k, m])
                    indices.append([I, J])

    return np.array(indices)

def compute_stencil(A, grid_size_x,grid_size_y,stencil_size):
    indices = get_indices_compute_A_one(grid_size_x,grid_size_y,stencil_size)
    stencil = np.array(A[indices[:, 0], indices[:, 1]]).reshape((grid_size_x, grid_size_y, stencil_size, stencil_size))
    return stencil
def get_indices_compute_A_one(grid_size_x=8,grid_size_y=8,stencil_size=3):
    indices = []
    K = map_2_to_1(grid_size_x,grid_size_y,stencil_size)
    for i in range(grid_size_x):
        for j in range(grid_size_y):
            I = int(K[i, j, stencil_size//2, stencil_size//2])
            for k in range(stencil_size):
                for m in range(stencil_size):
                    J = int(K[i, j, k, m])
                    indices.append([I, J])

    return np.array(indices)
def compute_p2(P_stencil, grid_size):
    indexes = get_p_matrix_indices_one(grid_size)
    P = sp.sparse.csr_matrix(arg1=(P_stencil.reshape(-1), (indexes[:, 1], indexes[:, 0])),
                   shape=((grid_size//2) ** 2, (grid_size) ** 2))

    return P
def compute_A_indices(grid_size_x=8,grid_size_y=8,stencil_size=3):
    K = map_2_to_1(grid_size_x,grid_size_y,stencil_size)
    A_idx = []
    stencil_idx = []
    for i in range(grid_size_x):
        for j in range(grid_size_y):
            I = int(K[i, j, stencil_size//2, stencil_size//2])
            for k in range(stencil_size):
                for m in range(stencil_size):
                    J = int(K[i, j, k, m])
                    A_idx.append([I, J])
                    stencil_idx.append([i, j, k, m])
    return np.array(A_idx), stencil_idx
def compute_p2_old(P_stencil, grid_size):
    indexes = get_p_matrix_indices_one_old(grid_size)
    P = sp.sparse.csr_matrix(arg1=(P_stencil.reshape(-1), (indexes[:, 1], indexes[:, 0])),
                   shape=((grid_size//2) ** 2, (grid_size) ** 2))

    return P
def get_p_matrix_indices_one_old(grid_size):
    K = map_2_to_1_old(grid_size=grid_size)
    indices = []
    for ic in range(grid_size // 2):
        i = 2 * ic + 1
        for jc in range(grid_size // 2):
            j = 2 * jc + 1
            J = int(grid_size // 2 * jc + ic)
            for k in range(3):
                for m in range(3):
                    I = int(K[i, j, k, m])
                    indices.append([I, J])

    return np.array(indices)
def map_2_to_1_old(grid_size=8):
    # maps 2D coordinates to the corresponding 1D coordinate in the matrix.
    k = np.zeros((grid_size, grid_size, 3, 3))
    M = np.reshape(np.arange(grid_size ** 2), (grid_size, grid_size))
    M = np.concatenate([M, M], 0)
    M = np.concatenate([M, M], 1)
    for i in range(3):
        I = (i - 1) % grid_size
        for j in range(3):
            J = (j - 1) % grid_size
            k[:, :, i, j] = M[I:I + grid_size, J:J + grid_size]
    return k

# def prolongation_fn(grid_size):
# #     grid_size = int(math.sqrt(A.shape[0]))
#     res_stencil = np.double(np.zeros((3,3)))
#     k=16
#     res_stencil[0,0] = 1/k
#     res_stencil[0,1] = 2/k
#     res_stencil[0,2] = 1/k
#     res_stencil[1,0] = 2/k
#     res_stencil[1,1] = 4/k
#     res_stencil[1,2] = 2/k
#     res_stencil[2,0] = 1/k
#     res_stencil[2,1] = 2/k
#     res_stencil[2,2] = 1/k
#     P_stencils= np.zeros((grid_size//2,grid_size//2,3,3))
#     for i in range(grid_size//2):
#         for j in range(grid_size//2):
#             P_stencils[i,j,:,:]=res_stencil
#     return compute_p2_old(P_stencils, grid_size).astype(np.double)  # imaginary part should be zero
def compute_A(P_stencil, grid_size_x,grid_size_y,stencil_size):
    A,indexes = compute_A_indices(grid_size_x,grid_size_y,stencil_size)
    P = torch.sparse.DoubleTensor(torch.LongTensor(A.T), P_stencil.view(-1), (grid_size_x*grid_size_y,grid_size_x*grid_size_y))
    return P
def compute_A_old(P_stencil, grid_size_x,grid_size_y,stencil_size):
    A,indexes = compute_A_indices(grid_size_x,grid_size_y,stencil_size)
    P = torch.sparse.DoubleTensor(torch.LongTensor(A.T), P_stencil.view(-1), (grid_size_x*grid_size_y,grid_size_x*grid_size_y))
    return P
def compute_A_numpy(P_stencil, grid_size_x,grid_size_y,stencil_size):
    A,indexes = compute_A_indices(grid_size_x,grid_size_y,stencil_size)
    P = csr_matrix(arg1=(P_stencil.reshape((-1)), (A[:, 0], A[:, 1])),
                              shape=(grid_size_x*grid_size_y,grid_size_x*grid_size_y))
    return P

def coo_to_tensor(coo):
    values = coo.data
    indices = np.vstack((coo.row, coo.col))
    i = torch.LongTensor(indices)
    v = torch.DoubleTensor(values)
    shape = coo.shape
    return torch.sparse.DoubleTensor(i, v, torch.Size(shape)).to(device)

In [ ]:
def elasticity(E=1e-5,nu=0.3,by=1):
    mu = E*(1+2*nu)
    lam = E*nu/((1+nu)*(1-2*nu))
    k17 = (-lam*by*by-2*mu*by*by+lam+mu)/(4*(2*lam*by*by+lam+4*(by*by+1)*mu))
    k14 = -(2*lam*by*by+4*mu*by*by+lam+mu)/(2*(2*lam*by*by+lam+4*(by*by+1)*mu))
    k11 = k17
    k12 = ((by**2-1)*lam+2*(by**2-2)*mu)/(2*(2*lam*by*by+lam+4*(by**2+1)*mu))
    k18 = k12
    k15 = 1
    k19 = k17
    k13 = k17
    k16 = k14
    s1 = np.array([[k17,k18,k19],[k14,k15,k16],[k11,k12,k13]])
    k = (3*by*(lam+mu))/(8*(2*lam*by*by+lam+4*(by*by+1)*mu))
    s2 = np.array([[k,0,-k],[0,0,0],[-k,0,k]])
    return s1,s2
def transpose_stencil_numpy(s):
    a = s[0,0]
    b = s[0,1]
    c = s[0,2]
    d = s[1,0]
    e = s[1,1]
    f = s[1,2]
    g = s[2,0]
    h = s[2,1]
    i = s[2,2]
    return np.array([[i,h,g],[f,e,d],[c,b,a]])

In [ ]:
def reorder(X,option):
    new_IDX = []
    old_IDX = []
    if len(X.shape)<3:
        if option == 1:
            m,n = X.shape
            Y = torch.zeros(m,n).double()
            IDX = []
            for i in range(m):
                for j in range(n):
                    if (i%2)==(j%2):
                        IDX.append((i,j))
            for (i,j) in IDX:
                new_i = (i+j)//2
                new_j = (-i+j)//2+m//2
                Y[new_i,new_j] = X[i,j]
                new_IDX.append(new_i*n+new_j)
        else:
            m,n = X.shape
            Y = torch.zeros(m,n).double()
            IDX = []
            for i in range(m):
                for j in range(n):
                    if (i%2)!=(j%2):
                        IDX.append((i,j))
            for (i,j) in IDX:
                new_i = (i+j)//2
                new_j = (-i+j)//2+m//2
                Y[new_i,new_j] = X[i,j]
                new_IDX.append(new_i*n+new_j)
    elif len(X.shape)==3:
        z,m,n = X.shape
        Y = torch.zeros(z,m,n).double()
        for i in range(z):
            Y[i,:,:] = reorder(X[i,:,:],1)[0]
    return Y,new_IDX,old_IDX
def reorder_T(X,option):
    new_IDX=[]
    old_IDX = []
    if option == 1:
        m,n = X.shape
        Y = torch.zeros(m,n).double()
        IDX = []
        for i in range(m):
            if i<=m//2:
                for j in range(n//2-i,n//2+i+1):
                    IDX.append((i,j))
            else:
                for j in range(n//2-m+i+1,n//2+m-i):
                    IDX.append((i,j))
        for (i,j) in IDX:
            new_i = (i-j)+m//2
            new_j = (i+j)-m//2
            Y[new_i,new_j] = X[i,j]
            new_IDX.append(new_i*n+new_j)
            old_IDX.append(i*n+j)
    elif option == 2:
        m,n = X.shape
        Y = torch.zeros(m,n).double()
        IDX = []
        for i in range(m):
            if i<m//2:
                for j in range(n//2-i-1,n//2+i+1):
                    IDX.append((i,j))
            elif i<(m//2)*2:
                for j in range(n//2-m+i+1,n//2+m-i-1):
                    IDX.append((i,j))

        for (i,j) in IDX:
            if 0<=i<m-1 and 0<=j<n-1:
                new_i = (i-j)+m//2
                new_j = (i+j)-m//2+1
                # print((i,j),(new_i,new_j))
                Y[new_i,new_j] = X[i,j]
                new_IDX.append(new_i*n+new_j)
                old_IDX.append(i*n+j)
    return Y,new_IDX,old_IDX

In [ ]:
res1_uu = np.zeros((3,3))
res1_uu[0,1] = .25
res1_uu[1,0] =  .25
res1_uu[1,1] =  1
res1_uu[1,2] =  .25
res1_uu[2,1] =  .25

res1_uv = np.zeros((3,3))
res1_uv[0,1] = .25
res1_uv[1,0] = -.25
res1_uv[1,1] = 0
res1_uv[1,2] = -.25
res1_uv[2,1] = .25

res1_vu = np.zeros((3,3))
res1_vu[0,1] = -.25
res1_vu[1,0] = .25
res1_vu[1,1] = 0
res1_vu[1,2] = .25
res1_vu[2,1] = -.25

res1_vv = np.zeros((3,3))
res1_vv[0,1] =.25
res1_vv[1,0] = .25
res1_vv[1,1] = 1
res1_vv[1,2] =.25
res1_vv[2,1] = .25
# res2_vv = np.zeros((3,3))
# res2_vv[0,0] = .25
# res2_vv[0,2] = .25
# res2_vv[1,1] = 1
# res2_vv[2,0] = .25
# res2_vv[2,2] = .25

In [ ]:
def coo_to_tensor(coo):
    values = coo.data
    indices = np.vstack((coo.row, coo.col))
    i = torch.LongTensor(indices)
    v = torch.DoubleTensor(values)
    shape = coo.shape
    temp = coo
    row = temp.row
    col = temp.col
    data = temp.data
    return torch.sparse_coo_tensor(i, v, torch.Size(shape),requires_grad = False)

In [ ]:
A_train1 = []
A_train2 = []
At_train1 = []
At_train2 = []
s_train = []
stencil_train1 = []
stencil_train2 = []
A_uu_train = []
A_uv_train = []
test_vec_uu_train = []
test_vec_vv_train = []
eig_vec_train = []
Ag_train = []
A0_train = []
u_train  = []
v_train  = []
n = 9
eps = 10
num_train = 2
nu = np.linspace(0.1,0.4,num_train)
for iii in range(num_train):
    s1,s2 = elasticity(E=1e-5,nu=nu[iii],by=1)
    s3 = transpose_stencil_numpy(s2)
    s4 = transpose_stencil_numpy(s1)
    Auu = pyamg.gallery.stencil_grid(s1,(n,n))
    Auv = pyamg.gallery.stencil_grid(s2,(n,n))
    Avu = pyamg.gallery.stencil_grid(s3,(n,n))
    Avv = pyamg.gallery.stencil_grid(s4,(n,n))

    A_uu_uv = sp.sparse.hstack((Auu,Auv))
    A_vu_vv = sp.sparse.hstack((Avu,Avv))
    A = sp.sparse.vstack((A_uu_uv,A_vu_vv))


    print('A.shape:',A.shape)
    R1_uu_train = pyamg.gallery.stencil_grid(res1_uu,(n,n)).tocsr()
    R1_uu = R1_uu_train
    P1_uu = R1_uu.T
    P1_uu_train = R1_uu_train.T

    R1_uv_train = pyamg.gallery.stencil_grid(res1_uv,(n,n)).tocsr()
    R1_uv = R1_uv_train
    P1_uv = R1_uv.T
    P1_uv_train = R1_uv_train.T

    R1_vu_train = pyamg.gallery.stencil_grid(res1_vu,(n,n)).tocsr()
    R1_vu = R1_vu_train
    P1_vu = R1_vu.T
    P1_vu_train = R1_vu_train.T

    R1_vv_train = pyamg.gallery.stencil_grid(res1_vv,(n,n)).tocsr()
    R1_vv = R1_vv_train
    P1_vv = R1_vv.T
    P1_vv_train = R1_vv_train.T

    R_uu_uv = sp.sparse.hstack((R1_uu,R1_uv))
    R_vu_vv = sp.sparse.hstack((R1_vu,R1_vv))
    R = sp.sparse.vstack((R_uu_uv,R_vu_vv)).tocsr()
    R = R[0:n*n*2:2,:]
    P = R.T
    R_uu_uv_train = sp.sparse.hstack((R1_uu_train,R1_uv_train))
    R_vu_vv_train = sp.sparse.hstack((R1_vu_train,R1_vv_train))
    R_train = sp.sparse.vstack((R_uu_uv_train,R_vu_vv_train))


    P_train = R_train.T
    A_train = R_train@A@P_train # why using R_train????
    print('A_train.shape:',A_train.shape)
    T = R_train@P_train
    A = R@A@P
    A_uu = A_train[0:n*n,0:n*n]
    A_uv = A_train[0:n*n,n*n:]
    A_vu = A_train[n*n:,0:n*n]
    A_vv = A_train[n*n:,n*n:]


    stencil_uu = compute_stencil(A_uu,n,n,7)
    stencil_uv = compute_stencil(A_uv,n,n,5)
    stencil_vu = compute_stencil(A_vu,n,n,5)
    stencil_vv = compute_stencil(A_vv,n,n,7)

    s_uu = stencil_uu[n//2,n//2,:,:]
    s_uu[abs(s_uu)<1e-15] = 0
    s_uv = stencil_uv[n//2,n//2,:,:]
    s_uv[abs(s_uv)<1e-15] = 0
    s_vu = stencil_vu[n//2,n//2,:,:]
    s_vu[abs(s_vu)<1e-15] = 0
    s_vv = stencil_vv[n//2,n//2,:,:]
    s_vv[abs(s_vv)<1e-15] = 0
    s_uu,_,_ = reorder(s_uu,1)
    # A_uu_conv =
    # s_uu = s_uu[2:5,1:6]
    s_uv,_,_ = reorder(s_uv,2)

    s_vu,_,_ = reorder(s_vu,2)

    s_vv,_,_ = reorder(s_vv,1)
    # s_vv = s_vv[2:5,1:6]

    Ag = coo_to_tensor(A.tocoo())
    A_uu_conv = nn.Conv2d(1, 1, 3, padding = 0, bias = False).double()
    s_uu = s_uu[1:6,1:6]
    A_uu_conv.weight = nn.Parameter(s_uu.view(1,1,5,5))
    A_vv_conv = nn.Conv2d(1, 1, 3, padding = 0, bias = False).double()
    s_vv = s_vv[1:6,1:6]
    A_vv_conv.weight = nn.Parameter(s_vv.view(1,1,5,5))

    A_uv_conv = nn.Conv2d(1, 1, 4, padding = 0, bias = False).double()
    s_uv = s_uv[0:4,0:4]
    A_uv_conv.weight = nn.Parameter(s_uv.view(1,1,4,4))
    A_vu_conv = nn.Conv2d(1, 1, 4, padding = 0, bias = False).double()
    s_vu = s_vu[0:4,0:4]
    A_vu_conv.weight = nn.Parameter(s_vu.view(1,1,4,4))
    A_uu_train.append(A_uu_conv)
    A_uv_train.append(A_uv_conv)

    k=10
    # eig_value,eig_vec = sp.sparse.linalg.eigs(A,k=k,which = 'SM')

    stencil_train = [stencil_uu,stencil_uv,stencil_vu,stencil_vv]
    # s_train = [s_uu,s_uv,s_vu,s_vv]
    s_train.append([s_uu,s_uv,s_vu])
    # eig_vec = eig_vec.T
    # eig_value,eig_vec = np.linalg.eig(A.toarray())
    eig_value,eig_vec = sp.sparse.linalg.eigs(A_train,M=T,k=k,which = 'SM')
    eig_vec = eig_vec.T
    eig_vec_uu = eig_vec[:,0:n*n].reshape(k,n,n)
    eig_vec_vv = eig_vec[:,n*n:].reshape(k,n,n)
    eig_vec_uu = torch.real(torch.from_numpy(eig_vec_uu))
    eig_vec_vv = torch.real(torch.from_numpy(eig_vec_vv))
    test_vec_uu = reorder(eig_vec_uu,1)[0].view(k,1,n,n)
    test_vec_vv = reorder(eig_vec_vv,1)[0].view(k,1,n,n)
    test_vec_uu_train.append(test_vec_uu)
    test_vec_vv_train.append(test_vec_vv)

    # print(test_vec_uu.shape)

    # A_train1.append(A1)
    # A_train2.append(A2)
    # At_train1.append(A1t)
    # At_train2.append(A2t)

    # stencil_train1.append(A1.weight)
    # stencil_train2.append(A2.weight)

    # eig_vec_train.append(eig_vec)
    # Ag_train.append(Ag)
    # A0_train.append(A0)
    # u_train.append(u)
    # v_train.append(v)

In [ ]:
def top_k(logits,k,tau=0):
    y_soft = logits.softmax(-1)
    index = torch.topk(y_soft,k)[1]
    y_hard = torch.zeros_like(logits, memory_format=torch.legacy_contiguous_format).scatter_(-1, index, 1.0)
    ret = y_hard - y_soft.detach() + y_soft

    return ret

In [ ]:
class GNN_prob(nn.Module):
    def __init__(self):
        super(GNN_prob, self).__init__()
        self.fc1 = nn.Linear(6,200).double().to(device)
        self.fc2 = nn.Linear(200,200).double().to(device)
        self.fc3 = nn.Linear(200,200).double().to(device)
        self.fc4 = nn.Linear(200,200).double().to(device)
        self.fc5 = nn.Linear(200,6).double().to(device)
        self.fc6 = nn.Linear(100,14).double().to(device)
        self.fc7 = nn.Linear(100,100).double().to(device)

        self.fc8 = nn.Linear(100,8).double().to(device)


        # torch.nn.init.sparse_(self.fc1.weight,0.1)
        # torch.nn.init.sparse_(self.fc2.weight,0.1)
        # torch.nn.init.sparse_(self.fc3.weight,0.15)
        # torch.nn.init.sparse_(self.fc4.weight,0.15)

    def forward(self,X):
        X = self.fc1(X)
        # X = torch.relu(X)

        X = torch.relu(X)
        # X = F.leaky_relu(X)

        X = self.fc2(X)
        X = torch.relu(X)

        # X = torch.relu(X)
        # X = F.leaky_relu(X)
        X = self.fc3(X)
        X = torch.relu(X)

        # X = torch.relu(X)
        # X = F.leaky_relu(X)
        X = self.fc4(X)
        X = torch.relu(X)
        # # X = torch.tanh(X)
        X = self.fc5(X)
        X = torch.relu(X)

        # # X = torch.tanh(X)
        # X = self.fc6(X)
        # X = torch.tanh(X)

        # X = self.fc7(X)
        # X = torch.tanh(X)

        # # X = F.leaky_relu(X)
        # X = self.fc8(X)
        # # X = torch.relu(X)
        # X = torch.tanh(X)
        # X = torch.tanh(X)

        # X = F.leaky_relu(X)

        return X
class GNN_value(nn.Module):
    def __init__(self):
        super(GNN_value, self).__init__()
        self.fc1 = nn.Linear(6,280).double().to(device)
        self.fc2 = nn.Linear(280,280).double().to(device)
        self.fc3 = nn.Linear(280,200).double().to(device)
        self.fc4 = nn.Linear(200,200).double().to(device)
        self.fc5 = nn.Linear(200,6).double().to(device)

        self.fc6 = nn.Linear(100,14).double().to(device)
        torch.nn.init.eye_(self.fc1.weight)
        torch.nn.init.eye_(self.fc2.weight)
        torch.nn.init.eye_(self.fc3.weight)
        torch.nn.init.eye_(self.fc4.weight)
        torch.nn.init.eye_(self.fc5.weight)
    def forward(self,X):
        X = self.fc1(X)
        # X = torch.relu(X)
        X = F.leaky_relu(X)

        X = self.fc2(X)
        # X = torch.relu(X)
        X = F.leaky_relu(X)
        X = self.fc3(X)
        X = F.leaky_relu(X)
        X = self.fc4(X)
        X = F.leaky_relu(X)
        X = self.fc5(X)
        X = F.leaky_relu(X)

        # X = torch.tanh(X)
        # X = self.fc5(X)

        # # X = torch.relu(X)
        # X = torch.tanh(X)
        # X = self.fc6(X)
        # X = F.leaky_relu(X)
        # X = torch.tanh(X)
        # X = F.leaky_relu(X)

        return X

def sparsify(prob,value,K,stencil_train):
    # prob = torch.sigmoid(X[0:8])
    # prob = X[0:8]
    prob = top_k(prob,K).squeeze()
    stencil = prob*value
    # stencil = value
    stencil = stencil.view(-1,1)
    stencil_uu = torch.from_numpy(stencil_train[0]).clone()
    stencil_uv = torch.from_numpy(stencil_train[1]).clone()
    stencil_vu = torch.from_numpy(stencil_train[2]).clone()
    stencil_vv = torch.from_numpy(stencil_train[3]).clone()
    m,n,_,_ = stencil_uu.shape
    s_uu = torch.zeros(5,5).double()
    ss_uu = torch.zeros(7,7).double()
    s_uv = torch.zeros(4,4).double()
    ss_uv = torch.zeros(5,5).double()

    s_vu = torch.zeros(4,4).double()
    ss_vu = torch.zeros(5,5).double()

    s_vv = torch.zeros(5,5).double()
    ss_vv = torch.zeros(7,7).double()

    s_uu[0,0] = 0
    s_uu[0,1] = stencil[0]
    s_uu[0,2] = 0
    s_uu[0,3] = stencil[1]
    s_uu[0,4] = 0
    s_uu[1,0] = stencil[1]
    s_uu[1,1] = stencil[2]
    s_uu[1,2] = stencil[3]
    s_uu[1,3] = stencil[4]
    s_uu[1,4] = stencil[0]
    s_uu[2,0] = 0
    s_uu[2,1] = stencil[3]
    s_uu[2,2] = -stencil[0]*4-stencil[1]*4-stencil[2]*2-4*stencil[3]-2*stencil[4]
    s_uu[2,3] = stencil[3]
    s_uu[2,4] = 0
    s_uu[3,0] = stencil[0]
    s_uu[3,1] = stencil[4]
    s_uu[3,2] = stencil[3]
    s_uu[3,3] = stencil[2]
    s_uu[3,4] = stencil[1]
    s_uu[4,0] = 0
    s_uu[4,1] = stencil[1]
    s_uu[4,2] = 0
    s_uu[4,3] = stencil[0]
    s_uu[4,4] = 0
    ss_uu[1:6,1:6] = s_uu

    s_vv[0,0] = 0
    s_vv[0,1] = stencil[1]
    s_vv[0,2] = 0
    s_vv[0,3] = stencil[0]
    s_vv[0,4] = 0
    s_vv[1,0] = stencil[0]
    s_vv[1,1] = stencil[2]
    s_vv[1,2] = stencil[3]
    s_vv[1,3] = stencil[4]
    s_vv[1,4] = stencil[1]
    s_vv[2,0] = 0
    s_vv[2,1] = stencil[3]
    s_vv[2,2] = -stencil[0]*4-stencil[1]*4-stencil[2]*2-4*stencil[3]-2*stencil[4]
    s_vv[2,3] = stencil[3]
    s_vv[2,4] = 0
    s_vv[3,0] = stencil[1]
    s_vv[3,1] = stencil[4]
    s_vv[3,2] = stencil[3]
    s_vv[3,3] = stencil[2]
    s_vv[3,4] = stencil[0]
    s_vv[4,0] = 0
    s_vv[4,1] = stencil[0]
    s_vv[4,2] = 0
    s_vv[4,3] = stencil[1]
    s_vv[4,4] = 0

    ss_vv[1:6,1:6] = s_vv

    s_uv[0,0] = 0
    s_uv[0,1] = stencil[5]
    s_uv[0,2] = stencil[5]
    s_uv[0,3] = 0
    s_uv[1,0] = -stencil[5]
    # s_uv[1,1] = stencil[6]
    # s_uv[1,2] = stencil[7]
    s_uv[1,3] = -stencil[5]
    s_uv[2,0] = -stencil[5]
    # s_uv[2,1] = stencil[8]
    # s_uv[2,2] = stencil[9]
    s_uv[2,3] = -stencil[5]
    s_uv[3,0] = 0
    s_uv[3,1] = stencil[5]
    s_uv[3,2] = stencil[5]
    s_uv[3,3] = 0
    ss_uv[0:4,0:4] = s_uv

    s_vu[0,0] = 0
    s_vu[0,1] = stencil[5]
    s_vu[0,2] = stencil[5]
    s_vu[0,3] = 0
    s_vu[1,0] = -stencil[5]
    # s_vu[1,1] = stencil[10]
    # s_vu[1,2] = stencil[11]
    s_vu[1,3] = -stencil[5]
    s_vu[2,0] = -stencil[5]
    # s_vu[2,1] = stencil[12]
    # s_vu[2,2] = stencil[13]
    s_vu[2,3] = -stencil[5]
    s_vu[3,0] = 0
    s_vu[3,1] = stencil[5]
    s_vu[3,2] = stencil[5]
    s_vu[3,3] = 0
    ss_vu[0:4,0:4] = s_vu



    ss_uu,_,_ = reorder_T(ss_uu.detach(),1)
    ss_uv,_,_ = reorder_T(ss_uv.detach(),2)
    ss_vu,_,_ = reorder_T(ss_vu.detach(),2)
    ss_vv,_,_ = reorder_T(ss_vv.detach(),1)

    for i in range(2,m-2):
        for j in range(2,n-2):
            stencil_uu[i,j,:,:] = ss_uu
            stencil_uv[i,j,:,:] = ss_uv
            stencil_vu[i,j,:,:] = ss_vu
            stencil_vv[i,j,:,:] = ss_vv
    # u,v = torch.topk(torch.sigmoid(prob),4)
    return stencil_uu,stencil_uv,stencil_vu,stencil_vv,s_uu,s_uv,s_vu,s_vv,ss_uu,ss_uv,ss_vu,ss_vv


def train(A_uu_train,A_uv_train,test_vec_uu_train,test_vec_vv_train,s_train,epochs,learning_rate,model_prob,model_value):

#     model = GCN(32,32,10)
    optimizer = torch.optim.Adam(list(model_prob.parameters())+list(model_value.parameters()),lr=learning_rate)
    # optimizer = torch.optim.SGD(list(model_prob.parameters())+list(model_value.parameters()), lr=learning_rate)

    tau = 1

    for epoch in range(epochs):
        loss = 0
        optimizer.zero_grad()

        for j in range(len(A_uv_train)):
            # for j in range(len(A_train1)):
            # s_uu,s_uv,s_vu,s_vv = s_train
            s_uu,s_uv,s_vu = s_train[j]
            A_uu_conv = A_uu_train[j]
            A_uv_conv = A_uv_train[j]
            test_vec_uu = test_vec_uu_train[j]
            test_vec_vv = test_vec_vv_train[j]
            n = 17

            # s_uv = torch.from_numpy(s_uv).view(-1,1).clone()
            # s_vu = torch.from_numpy(s_vu).view(-1,1).clone()
            # s_vv = torch.from_numpy(s_vv).view(-1,1).clone()
            # stencil_uu_train = torch.cat([s_uu[0:24],s_uu[25:49]])
            # stencil_uv_train = torch.cat([s_uv[0:24],s_uv[25:49]])
            # stencil_vu_train = torch.cat([s_vu[0:24],s_vu[25:49]])
            # stencil_vv_train = torch.cat([s_vv[0:24],s_vv[25:49]])
            # stencil = torch.cat([stencil_uu_train,stencil_uv_train,stencil_vu_train,stencil_vv_train]).view(1,-1)
            stencil = torch.zeros(1,6).double()
            stencil[0,0] = s_uu[0,1]
            stencil[0,1] = s_uu[0,3]
            stencil[0,2] = s_uu[1,1]
            stencil[0,3] = s_uu[1,2]
            stencil[0,4] = s_uu[1,3]
            stencil[0,5] = s_uv[0,1]
            # stencil[0,6] = s_uv[1,1]
            # stencil[0,7] = s_uv[1,2]
            # stencil[0,8] = s_uv[2,1]
            # stencil[0,9] = s_uv[2,2]
            # stencil[0,10] = s_vu[1,1]
            # stencil[0,11] = s_vu[1,2]
            # stencil[0,12] = s_vu[2,1]
            # stencil[0,13] = s_vu[2,2]

            # stencil[0,7] = s_uu[3,3]
            # stencil[0,8] = s_uu[3,4]
            # stencil[0,9] = s_uu[3,5]
            # stencil[0,10] = s_uu[4,1]
            # stencil[0,11] = s_uu[4,2]
            # stencil[0,12] = s_uu[4,3]
            # stencil[0,13] = s_uu[4,4]
            # stencil[0,14] = s_uu[4,5]

            prob = model_prob(stencil.to(device)).squeeze()
            # print(stencil)
            value = model_value(stencil.to(device)).squeeze()
            # print(value)
            # print(stencil)
            # value = stencil
            stencil_uu,stencil_uv,stencil_vu,stencil_vv,s_uu,s_uv,s_vu,s_vv,ss_uu,ss_uv,ss_vu,ss_vv = sparsify(prob,value,int(len(prob)*0.5),stencil_train)
            temp_uu = F.conv2d(test_vec_uu,s_uu.view(1,1,5,5),padding=0)-A_uu_conv(test_vec_uu)
            temp_uv = F.conv2d(test_vec_vv,s_uv.view(1,1,4,4),padding=0)-A_uv_conv(test_vec_vv)

            # temp = ss_uu-A_uu_conv.weight.squeeze()
            loss += torch.norm(temp_uu)**2+torch.norm(temp_uv)**2
        loss.backward()
        optimizer.step()

        # temp = torch.mm(A_c,eig_vec)-torch.mm(Ag.to_dense(),eig_vec)
        # # print(temp)
        # # temp = temp.squeeze(1).view(temp.shape[0],-1,1)
        # loss+=torch.norm(temp)**2
        # loss.backward()

        # for name, param in model_value.named_parameters():
        #     print(name, param.grad)

        if epoch%100==0:
            print(' epoch: ',epoch,' loss: ',loss)
    A_uu = compute_A(stencil_uu,stencil_uu.shape[0],stencil_uu.shape[1],stencil_uu.shape[2]).to_dense()
    A_uv = compute_A(stencil_uv,stencil_uv.shape[0],stencil_uv.shape[1],stencil_uv.shape[2]).to_dense()
    A_vu = compute_A(stencil_vu,stencil_vu.shape[0],stencil_vu.shape[1],stencil_vu.shape[2]).to_dense()
    A_vv = compute_A(stencil_vv,stencil_vv.shape[0],stencil_vv.shape[1],stencil_vv.shape[2]).to_dense()
    A_uu_uv = torch.hstack((A_uu,A_uv))
    A_vu_vv = torch.hstack((A_vu,A_vv))
    A_c = torch.vstack((A_uu_uv,A_vu_vv))
    A_c = A_c[0:n*n*2:2,:]
    A_c = A_c[:,0:n*n*2:2]

    return model_prob,model_value,A_c,ss_uu,ss_vv,ss_uv,ss_vu

In [ ]:
# n = 31
# num_eigenvecs = n*n
# eig_vec = torch.rand(num_eigenvecs,1,n,n).double()
# b = torch.rand(num_eigenvecs,1,n,n).doubl()
# for k in range(10):
#   # eig_vec = np.matmul(np.linalg.inv(L.toarray()),b-np.matmul(U.toarray(),eig_vec))
#   eig_vec = 2/(3*A0.weight[0,0,1,1])*(b-A0(eig_vec))+eig_vec
# eig_vec = b-A0(eig_vec)
# eig_vec = res(eig_vec)
model_prob = GNN_prob()
model_value = GNN_value()
mxl = 1
for i in range(mxl):
    model_prob,model_value,A_c,ss_uu,ss_vv,ss_uv,ss_vu = train(A_uu_train,A_uv_train,test_vec_uu_train,test_vec_vv_train,s_train,80,1e-5,model_prob,model_value)
    # print(coarse_stencil)
    # A = pyamg.gallery.stencil_grid(coarse_stencil.squeeze().detach().numpy(),(n,n)).tocsr()
    # A0 = coo_to_tensor(A.tocoo())
    # k=10

    # R = prolongation_fn(n)
    # P = R.T
    # T = R*P

    # A=R*A*P

    # Ag = coo_to_tensor(A.tocoo())
    # eig_value,eig_vec = sp.sparse.linalg.eigs(A,k=k,M=T,which = 'SM')
    # eig_vec = eig_vec.T
    # eig_vec = torch.real(torch.from_numpy(eig_vec)).view(k,1,n//2,n//2)
    # A = project_stencil(coarse_stencil.view(1,1,3,3))
    # A_train.append(A)
    # stencil_train.append(A.weight)
    # eig_vec_train.append(eig_vec)
    # Ag_train.append(Ag)
    # A0_train.append(A0)
    # print('level '+str(i)+' training done! ')

In [ ]:
def geometric_solver(A,option1,option2,models,n,
                     presmoother=('gauss_seidel', {'sweep': 'forward'}),
                     postsmoother=('gauss_seidel', {'sweep': 'forward'}),
                     max_levels=5, max_coarse=10,coarse_solver='splu',stencil=0,**kwargs):

    levels = [multilevel_solver.level()]

    # convert A to csr
    if not isspmatrix_csr(A):
        try:
            A = csr_matrix(A)
            warn("Implicit conversion of A to CSR",
                 SparseEfficiencyWarning)
        except BaseException:
            raise TypeError('Argument A must have type csr_matrix, \
                             or be convertible to csr_matrix')
    # preprocess A
    A = A.asfptype()
    if A.shape[0] != A.shape[1]:
        raise ValueError('expected square matrix')

    levels[-1].A = A
    levels[-1].stencil = stencil
    levels[-1].n = n

    while len(levels) < max_levels and levels[-1].A.shape[0] > max_coarse:
        extend_hierarchy(levels,option1,option2,models,stencil)

    ml = multilevel_solver(levels, **kwargs)
    change_smoothers(ml, presmoother, postsmoother)
    return ml

# internal function
def extend_hierarchy(levels,option1,option2,models,stencil):
    """Extend the multigrid hierarchy."""
    if len(levels) == 1:
        A = levels[-1].A
        n = levels[-1].n
        model = models[len(levels)-1]
        R1_uu_train = pyamg.gallery.stencil_grid(res1_uu,(n,n)).tocsr()
        # R1_uu_train[idx,:] = 0
        R1_uu = R1_uu_train
        I = sp.sparse.csr_matrix(R1_uu.shape)
        P1_uu = R1_uu.T
        R1_uv_train = pyamg.gallery.stencil_grid(res1_uv,(n,n)).tocsr()

        R1_uv = R1_uv_train
        P1_uv = R1_uv.T
        R1_vu_train = pyamg.gallery.stencil_grid(res1_vu,(n,n)).tocsr()

        R1_vu = R1_vu_train
        P1_vu = R1_vu.T
        R1_vv_train = pyamg.gallery.stencil_grid(res1_vv,(n,n)).tocsr()

        R1_vv = R1_vv_train
        P1_vv = R1_vv.T
        R_uu_uv = sp.sparse.hstack((R1_uu,R1_vu))
        R_vu_vv = sp.sparse.hstack((R1_uv,R1_vv))

        R_train = sp.sparse.vstack((R_uu_uv,R_vu_vv)).tocsr()
        R = R_train[0:n*n*2:2,:]
        P = R.T

        if option1=='standard':
            # Form next level through Galerkin product        # print(A)
            print(R.shape)
            print(A.shape)
            A = R*A*P
            # eig_value,eig_vec = sp.sparse.linalg.eigs(A,feature_dims,which = 'SM')
            # X = torch.real(torch.from_numpy(eig_vec).to(device))
            n=n//2
            levels[-1].P = P  # prolongation operator
            levels[-1].R = R  # restriction operator

            levels.append(multilevel_solver.level())
            A = A.astype(np.float64)  # convert from complex numbers, should have A.imag==0
            levels[-1].A = A.tocsr()
            levels[-1].n = n
        else:
            if len(levels) == 1:
                A_train = R_train*A*R_train.T
                A_uu = A_train[0:n*n,0:n*n]
                A_uv = A_train[0:n*n,n*n:]
                A_vu = A_train[n*n:,0:n*n]
                A_vv = A_train[n*n:,n*n:]

                stencil_uu = compute_stencil(A_uu,n,n,7)
                stencil_uv = compute_stencil(A_uv,n,n,5)
                stencil_vu = compute_stencil(A_vu,n,n,5)
                stencil_vv = compute_stencil(A_vv,n,n,7)
                stencil_test = [stencil_uu,stencil_uv,stencil_vu,stencil_vv]

                s_uu = torch.from_numpy(stencil_uu[n//2,n//2,:,:])
                s_uv = torch.from_numpy(stencil_uv[n//2,n//2,:,:])
                stencil = torch.zeros(1,6).double()
                stencil[0,0] = s_uu[0,1]
                stencil[0,1] = s_uu[0,3]
                stencil[0,2] = s_uu[1,1]
                stencil[0,3] = s_uu[1,2]
                stencil[0,4] = s_uu[1,3]
                stencil[0,5] = s_uv[0,1]

                prob = model_prob(stencil.to(device)).squeeze()
                value = model_value(stencil.to(device)).squeeze()
                stencil_uu,stencil_uv,stencil_vu,stencil_vv,s_uu,s_uv,s_vu,s_vv,ss_uu,ss_uv,ss_vu,ss_vv = sparsify(prob,value,int(len(prob)*0.5),stencil_test)

                A_uu = compute_A_numpy(stencil_uu,stencil_uu.shape[0],stencil_uu.shape[1],stencil_uu.shape[2])
                A_uv = compute_A_numpy(stencil_uv,stencil_uv.shape[0],stencil_uv.shape[1],stencil_uv.shape[2])
                A_vu = compute_A_numpy(stencil_vu,stencil_vu.shape[0],stencil_vu.shape[1],stencil_vu.shape[2])
                A_vv = compute_A_numpy(stencil_vv,stencil_uu.shape[0],stencil_uu.shape[1],stencil_uu.shape[2])
                A_uu_uv = sp.sparse.hstack((A_uu,A_uv))
                A_vu_vv = sp.sparse.hstack((A_vu,A_vv))
                A_c = sp.sparse.vstack((A_uu_uv,A_vu_vv)).tocsr()
                A_c = A_c[0:n*n*2:2,:]
                A_c = A_c[:,0:n*n*2:2]

                levels[-1].P = P  # prolongation operator
                levels[-1].R = R  # restriction operator
                n=n//2
                levels.append(multilevel_solver.level())
                # stencil = project_stencil(stencil).weight

                # s = np.array([[0,-1,0],[-1,4,-1],[0,-1,0]])/16**(len(levels)-1)
                levels[-1].A = A_c
        #         print(A_c.shape)
                print('naive',A_c.count_nonzero()/(R*A*P).count_nonzero())
                # print(' theta ',np.linalg.norm(np.identity(A_c.shape[0])-np.matmul(np.linalg.inv(A_c.toarray()),(R*A*P).toarray()),2))
            else:
                A = R*A*P
                # eig_value,eig_vec = sp.sparse.linalg.eigs(A,feature_dims,which = 'SM')
                # X = torch.real(torch.from_numpy(eig_vec).to(device))
                n=n//2
                levels[-1].P = P  # prolongation operator
                levels[-1].R = R  # restriction operator

                levels.append(multilevel_solver.level())
                A = A.astype(np.float64)  # convert from complex numbers, should have A.imag==0
                levels[-1].A = A.tocsr()

        levels[-1].n=n
    else:
        A = levels[-1].A
        C = pyamg.strength.classical_strength_of_connection(A.tocsr())
        # C = algebraic_distance(A.tocsr())
        def unpack_arg(v):
            if isinstance(v, tuple):
                return v[0], v[1]
            else:
                return v, {}
        flag, kwargs = unpack_arg(False)
        AggOp = standard_aggregation(C, **kwargs)[0]
        # AggOp = naive_aggregation(C, **kwargs)[0]

        #AggOp =   lloyd_aggregation(C, **kwargs)[0]

        B = np.kron(np.ones((int(A.shape[0]/pyamg.util.utils.get_blocksize(A)), 1), dtype=A.dtype),
                  np.eye(pyamg.util.utils.get_blocksize(A), dtype=A.dtype))
        T, B = fit_candidates(AggOp, B)
        # #P = jacobi_prolongation_smoother(A, T, C, B, **kwargs)
        # P = richardson_prolongation_smoother(A,T,**kwargs)
        # T = prolongation_fn(int(np.sqrt(A.shape[0]))).T
        P = jacobi_prolongation_smoother(A, T, C, np.ones(T.shape[1]))
        R = P.T
        levels[-1].P = P  # prolongation operator
        levels[-1].R = R  # restriction operator
        levels.append(multilevel_solver.level())
        # stencil = project_stencil(stencil).weight

        # s = np.array([[0,-1,0],[-1,4,-1],[0,-1,0]])/16**(len(levels)-1)
        levels[-1].A = R*A*P
    print(str(len(levels)),levels[-1].A.tocsr()[levels[-1].A.tocsr().shape[0]//2,:].count_nonzero())

In [ ]:
def fx(x,y,nu):
    A1 = (-66+82*nu)*np.cos(np.pi*(5*x+4*y))
    A2 = -21*np.cos(np.pi*3*x)*np.sin(7*np.pi*y)
    return np.pi*np.pi*(A1+A2)/(2*(-1+nu+2*nu*nu))
def fy(x,y,nu):
    A1 = -20*np.cos(np.pi*(5*x+4*y))
    A2 = (-107+116*nu)*np.cos(np.pi*7*y)*np.sin(3*np.pi*x)
    return np.pi*np.pi*(A1+A2)/(2*(-1+nu+2*nu*nu))


In [ ]:
# torch.set_printoptions(precision=10)
num_test = 1
t1=time.time()
num_iter = []
num_iter2 = []
num_iter3 = []
num_iter4 = []
t_iter = []
t_iter2 = []
t_iter3 = []
t_iter4 = []
res_s = []
res2_s = []
res3_s = []
res4_s = []
test_grid_size = 129
model1=0
model2=0
model3=0

for i in range(num_test):
    nu = np.random.rand()*0.09+0.211
    s1,s2 = elasticity(E=1e-5,nu=nu,by=1)
    s3 = transpose_stencil_numpy(s2)
    s4 = transpose_stencil_numpy(s1)
    x = np.linspace(0, 1, test_grid_size)
    y = np.linspace(0, 1, test_grid_size)
    xv, yv = np.meshgrid(x, y)
    uu = fx(xv,yv,0.3).reshape(-1,1)
    vv = fy(xv,yv,0.3).reshape(-1,1)
    b = np.vstack((uu,vv))
    Auu = pyamg.gallery.stencil_grid(s1,(test_grid_size,test_grid_size)).tocsr()
    Auv = pyamg.gallery.stencil_grid(s2,(test_grid_size,test_grid_size)).tocsr()
    Avu = pyamg.gallery.stencil_grid(s3,(test_grid_size,test_grid_size)).tocsr()
    Avv = pyamg.gallery.stencil_grid(s4,(test_grid_size,test_grid_size)).tocsr()

    A_uu_uv = sp.sparse.hstack((Auu,Auv))
    A_vu_vv = sp.sparse.hstack((Avu,Avv))
    A_orig = sp.sparse.vstack((A_uu_uv,A_vu_vv)).tocsr()


    solver_standard = geometric_solver(A_orig,'standard',0,[model1,model2,model3],test_grid_size,max_levels=2,coarse_solver='splu')
    solver_non_galerkin = geometric_solver(A_orig,'non-galerkin','GNN',[model1,model2,model3],test_grid_size,max_levels=2,coarse_solver='splu', stencil =0)
    # solver_non_galerkin_NONGNN = geometric_solver(A_orig,'234','123',[model1,model2,model3],test_grid_size,max_levels=2,coarse_solver='splu',stencil = torch.from_numpy(s).reshape(1,1,3,3))

    x0 = np.ones((A_orig.shape[0],1))
    # b = np.zeros((A_orig.shape[0],1))

    res=[]
    res2= []
    res3= []
    res4 = []
    t1 =time.time()
    x1 = solver_standard.solve(b,x0=x0,maxiter=1000, tol=1e-6,residuals=res)
    t2=time.time()
    x2 = solver_non_galerkin.solve(b,x0=x0,maxiter=1000, tol=1e-6,residuals=res2)
    t3=time.time()
    # x = solver_non_galerkin_NONGNN.solve(b,x0=x0,maxiter=1000, tol=1e-6,residuals=res3)
    # t4=time.time()
    # x = solver_naive.solve(b,x0=x0,maxiter=1000, tol=1e-6,residuals=res4)
    # t5=time.time()

    res_s.append(res)
    res2_s.append(res2)
    # res3_s.append(res3)
    # res4_s.append(res4)

    num_iter.append(len(res))
    num_iter2.append(len(res2))
    # num_iter3.append(len(res3))
    # num_iter4.append(len(res4))

    t_iter.append(t2-t1)
    t_iter2.append(t3-t2)
    # t_iter3.append(t4-t3)
    # t_iter4.append(t5-t4)


print('standard iter:   ',np.mean(num_iter),'  standard time:    ',np.mean(t_iter))
print('non galerkin iter:   ',np.mean(num_iter2),'  non galerkin time:    ',np.mean(t_iter2))
# print('non galerkin nongnn iter:   ',np.mean(num_iter3),'  non galerkin time:    ',np.mean(t_iter3))
# print('naive iter:   ',np.mean(num_iter4),'  non galerkin time:    ',np.mean(t_iter4))


In [ ]:
Ag = solver_standard.levels[-1].A
Ac = solver_non_galerkin.levels[-1].A

# Ag_eig = sp.linalg.eigvals(Ag.toarray())
# Ac_eig = sp.linalg.eigvals(Ac.toarray())
test_vec = np.matmul(Ac.toarray(),sp.linalg.inv(Ag.toarray()))
eig1 = sp.linalg.eigvals(test_vec)

In [ ]:
Ag = solver_standard.levels[-1].A
Ac = solver_non_galerkin.levels[-1].A

# Ag_eig = sp.linalg.eigvals(Ag.toarray())
# Ac_eig = sp.linalg.eigvals(Ac.toarray())
test_vec = np.matmul(Ac.toarray(),sp.linalg.inv(Ag.toarray()))
eig2 = sp.linalg.eigvals(test_vec)

In [ ]:
Ag = solver_standard.levels[-1].A
Ac = solver_non_galerkin.levels[-1].A

# Ag_eig = sp.linalg.eigvals(Ag.toarray())
# Ac_eig = sp.linalg.eigvals(Ac.toarray())
test_vec = np.matmul(Ac.toarray(),sp.linalg.inv(Ag.toarray()))
eig3 = sp.linalg.eigvals(test_vec)

In [ ]:
Ag = solver_standard.levels[-1].A
Ac = solver_non_galerkin.levels[-1].A

# Ag_eig = sp.linalg.eigvals(Ag.toarray())
# Ac_eig = sp.linalg.eigvals(Ac.toarray())
test_vec = np.matmul(Ac.toarray(),sp.linalg.inv(Ag.toarray()))
eig4 = sp.linalg.eigvals(test_vec)

In [ ]:
# tt = np.identity(test_vec.shape[0])-test_vec
# print(np.linalg.norm(tt,2))

In [ ]:
print(len(eig4))
print(len(eig3))

In [ ]:
import matplotlib.pyplot as plt

t1 = np.linspace(1, len(eig1), len(eig1))
t2 = np.linspace(1, len(eig1), len(eig2))
t3 = np.linspace(1, len(eig1), len(eig3))
t4 = np.linspace(1, len(eig1), len(eig4))

fig, ax = plt.subplots(1, 1)
plt.plot(t1, np.sort(np.real(eig1)), 'r-',label='n=129')
plt.plot(t2, np.sort(np.real(eig2)), 'g',label='n=65')
plt.plot(t3, np.sort(np.real(eig3)), 'b--',label='n=33')
plt.plot(t4, np.sort(np.real(eig4)), 'y+',label='n=17')

plt.legend(fontsize=14)
# plt.legend('errors H','errors jacobi')
plt.ylabel('Eigenvalues',fontsize=18)
plt.xlabel('Index',fontsize=18)




# ax.set_xticks([15,30,45,60,75])
# plt.plot(t, np.sort(np.real(Ag_eig)), 'r-',label=r'$A_{g}$')
# # plt.plot(t, t2, 'g',label='Gauss-Siedel Smoother')
# plt.plot(t, np.sort(np.real(Ac_eig)), 'b--',label=r'$A_{c}$')
# plt.legend(fontsize=14)
# # plt.legend('errors H','errors jacobi')
# plt.ylabel('Eigenvalues',fontsize=18)
# plt.xlabel('Index',fontsize=18)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
def heatmap2d(arr: np.ndarray,name):
    plt.imshow(arr, cmap='viridis')
    #plt.colorbar()
    plt.axis('off')
    # plt.savefig(name)
    plt.show()
heatmap2d(x1[0:test_grid_size*test_grid_size].reshape(test_grid_size,test_grid_size),'cylinder')
heatmap2d(x1[test_grid_size*test_grid_size:].reshape(test_grid_size,test_grid_size),'cylinder')
heatmap2d(x2[0:test_grid_size*test_grid_size].reshape(test_grid_size,test_grid_size),'cylinder')
heatmap2d(x2[test_grid_size*test_grid_size:].reshape(test_grid_size,test_grid_size),'cylinder')
# print(np.linalg.norm(x1[0:test_grid_size*test_grid_size].reshape(test_grid_size,test_grid_size)-x1[test_grid_size*test_grid_size:].reshape(test_grid_size,test_grid_size)))